# Lesson 5 - Assignment

In this assignment, you will implement a Support Vector Machine Classifier  from scratch and compare the results to existing sklearn algorithm. 

In [1]:
# import packages
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from matplotlib.legend_handler import HandlerLine2D
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

# make this notebook's output stable across runs
np.random.seed(0)

Question 1.1: Implement the cost function cost/objective function:
<img src="https://miro.medium.com/max/688/1*JAS6rUTO7TDlrv4XZSMbsA.png" alt="drawing" width="600"/>


Hints given: 
There was some confusion about what the stochastic descent cost gradient function reg_strength parameter means, 
as in the assignment there is not a lot of context around it. reg_strength refers to the regulation coefficient in stochastic descent. 
reg_strength corresponds to L2 regularization, also known as Ridge regression. 
The regularization parameter (λ) controls the strength of the penalty. 
A higher λ will lead to smaller coefficients and potentially prevent overfitting. 
To illustrate how it is used, here is a general example of computing the cost gradient that incorporates reg_strength:

 
```
import numpy as np

def stochastic_descent_gradient(X, y, theta, learning_rate, reg_strength, batch_size):
    """
    Performs one epoch of stochastic gradient descent with L2 regularization.

    Args:
        X (np.ndarray): Feature matrix of shape (n_samples, n_features).
        y (np.ndarray): Target vector of shape (n_samples,).
        theta (np.ndarray): Parameter vector of shape (n_features,).
        learning_rate (float): Learning rate for gradient descent.
        reg_strength (float): Regularization strength (lambda).
        batch_size (int): Number of samples in each mini-batch.

    Returns:
        np.ndarray: Updated parameter vector.
    """
    n_samples = X.shape[0]
    indices = np.random.permutation(n_samples)
    
    for start_idx in range(0, n_samples, batch_size):
        end_idx = min(start_idx + batch_size, n_samples)
        batch_indices = indices[start_idx:end_idx]
        X_batch = X[batch_indices]
        y_batch = y[batch_indices]
        
        # Compute predictions
        predictions = X_batch.dot(theta)
        
        # Compute gradient of the cost function with regularization
        error = predictions - y_batch
        gradient = (X_batch.T.dot(error) / batch_size) + reg_strength * theta
        
        # Update parameters
        theta -= learning_rate * gradient

    return theta
```

Relationships among the Lesson 4 assignment methods

The question came up what are the relationships among the Lesson 4 assignment methods that work together to perform stochastic gradient descent.

```calculate_cost_gradient(W, X_batch, Y_batch, reg_strength)```

Computes the cost gradient of the loss function (including regularization) for a specific data batch (X_batch, Y_batch) using L2 regularization strength reg_strength and current weights W.
Used inside sgd to obtain the gradient for updating W.
```compute_cost(W, X, Y, reg_strength)```

Calculates the total or full dataset cost (including regularization) for the entire dataset (X, Y) using current weights W.
Typically called within sgd every few epochs.
```sgd(data, outputs, learning_rate, max_epochs)```

Implements the main training loop:
Calls calculate_cost_gradient to get the gradient
Updates W using the gradient and learning rate.
Calls compute_cost(W, X, Y, reg_strength) periodically to do a convergence check



In [2]:
def compute_cost(W, X, Y, reg_strength=1000):
    # number of training examples
    N = X.shape[0]
    
    # The labels in Y are 0 or 1. For SVM, we need -1 or 1.
    Y_svm = Y.copy()
    if hasattr(Y_svm, 'values'): # Handle pandas Series
        Y_svm = Y_svm.values
    Y_svm[Y_svm == 0] = -1

    # calculate hinge loss
    distances = 1 - Y_svm * (X.dot(W))
    hinge_loss_per_sample = np.maximum(0, distances)
    total_hinge_loss = np.sum(hinge_loss_per_sample) / N
    
    # calculate cost
    # Regularization term (don't regularize bias)
    # Bias is the last weight since intercept is the last column in X
    regularization = (1 / 2) * np.dot(W[:-1], W[:-1])
    
    cost = regularization + reg_strength * total_hinge_loss
    
    return cost

Question 1.2: Write a method that calculate the cost gradient:
<img src="https://miro.medium.com/max/866/1*ww3F21VMVGp2NKhm0VTesA.png" alt="drawing" width="600"/>

In [3]:
def calculate_cost_gradient(W, X_batch, Y_batch, reg_strength=1000):
    # The labels in Y_batch are 0 or 1. For SVM, we need -1 or 1.
    Y_svm = Y_batch.copy()
    if hasattr(Y_svm, 'values'): # Handle pandas Series
        Y_svm = Y_svm.values
    Y_svm[Y_svm == 0] = -1

    # number of training examples in batch
    N = X_batch.shape[0]

    # y_i * (w . x_i)
    margin = Y_svm * (X_batch.dot(W))
    
    # find indices of support vectors (where margin < 1)
    sv_indices = np.where(margin < 1)[0]
    
    # Gradient of the loss term
    X_batch_np = X_batch.values if hasattr(X_batch, 'values') else X_batch
    dw_loss = -np.dot(Y_svm[sv_indices], X_batch_np[sv_indices, :])
    dw_loss = reg_strength * (dw_loss / N)
    
    # Gradient of the regularization term (don't regularize bias)
    dw_reg = W.copy()
    dw_reg[-1] = 0
    
    # Total gradient
    dw = dw_reg + dw_loss
    
    return dw

Question 1.3: Write a method that performs stochastic Gradient descent as follows:
- Caluclate the gradient of cost function i.e. ∇J(w)
- Update the weights in the opposite direction to the gradient: w = w — ∝(∇J(w))
- Repeat until conversion or until 5000 epochs are reached

In [4]:
def sgd(data, outputs, learning_rate=0.0001, max_epochs=5000):
    # Initialize weights
    n_features = data.shape[1]
    weights = np.zeros(n_features)
    
    prev_cost = float("inf")
    
    # Set batch size
    batch_size = 100
    
    print("training started...")
    
    # Epoch loop
    for epoch in range(1, max_epochs + 1):
        
        X_shuffled, y_shuffled = shuffle(data, outputs, random_state=epoch)
        
        for i in range(0, X_shuffled.shape[0], batch_size):
            X_batch = X_shuffled.iloc[i:i+batch_size]
            y_batch = y_shuffled.iloc[i:i+batch_size]
            
            grad = calculate_cost_gradient(weights, X_batch, y_batch)
            weights = weights - learning_rate * grad
            
        # Print cost to monitor
        if epoch <= 10 or epoch % 100 == 0:
            cost = compute_cost(weights, data, outputs)
            print(f"Epoch is:{epoch} and Cost is: {cost}")
            if abs(prev_cost - cost) < 1e-4:
                print("Converged.")
                break
            prev_cost = cost

    print("training finished.")
    return weights

# Dataset

In [5]:
data = pd.read_csv('data_banknote_authentication.csv')

Y = data.iloc[:, -1]  
X = data.iloc[:, 1:4]
X.insert(loc=len(X.columns), column='intercept', value=1)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4, random_state=42)

Question 4: Train and evaluate an SVC using the banknote_authentication data

In [6]:
# train the model
W = sgd(X_train, y_train, learning_rate=0.001, max_epochs=1000)

print(f"weights are: {W}")

# make predictions on the test set
y_pred_svm = X_test.dot(W)
y_pred = np.where(y_pred_svm > 0, 1, 0)

# calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"accuracy on test dataset: {accuracy}")

training started...
Epoch is:1 and Cost is: 977.898852406224
Epoch is:2 and Cost is: 3258.5389430978903
Epoch is:3 and Cost is: 3772.381001174172
Epoch is:4 and Cost is: 1086.6774316794063
Epoch is:5 and Cost is: 3432.2854365387175
Epoch is:6 and Cost is: 1483.2441852517854
Epoch is:7 and Cost is: 4670.316618068469
Epoch is:8 and Cost is: 928.3480668454793
Epoch is:9 and Cost is: 4512.34754163318
Epoch is:10 and Cost is: 1530.6014595407223


Epoch is:100 and Cost is: 636.6306408749992


Epoch is:200 and Cost is: 2932.3810921842087


Epoch is:300 and Cost is: 2362.0498807055756


Epoch is:400 and Cost is: 1599.0683641766177


...

Epoch is:600 and Cost is: 1154.4943047382014


Epoch is:700 and Cost is: 2515.050013057047


Epoch is:800 and Cost is: 781.7343757717199


Epoch is:900 and Cost is: 616.9904782562888


Epoch is:1000 and Cost is: 2464.5569658367604
training finished.
weights are: [-3.10348684 -0.69805664 -1.20203299  1.49363636]
accuracy on test dataset: 0.6593806921675774


[Bonus] Question 5: Train and evaluate an SKLEARN SVC model, and compare the results to your model 

In [7]:
from sklearn.svm import SVC

# Create and train the model
# Using a linear kernel to be comparable to our implementation
svc_model = SVC(kernel='linear', random_state=0)
svc_model.fit(X_train, y_train)

# Make predictions
y_pred_sklearn = svc_model.predict(X_test)

# Calculate and print accuracy
accuracy_sklearn = accuracy_score(y_test, y_pred_sklearn)
print(f"SKLEARN SVC accuracy on test dataset: {accuracy_sklearn}")

SKLEARN SVC accuracy on test dataset: 0.8142076502732241
